In [1]:
"""
MYSQL DATA ANALYSIS


To successfully follow this notebook, 
you will need to first install mysql on your laptop and configure it appropriately. 
You can follow the guide at: 
https://dev.mysql.com/downloads/mysql/
https://dev.mysql.com/doc/refman/8.0/en/binary-installation.html#binary-installation-createsysuser
Then install mysql on python using pip

NOTE: You must have your mysql server running before you can access it in python

This is data analysis using Structured Query Language (SQL)
SQL is a tool for every data scientist for getting access to data, 
and for advance data analysis.
Designing the data stracture properly is very important, 
it is adversable to devide data to many tables, and not stored all in one place


The notebook will cover:
Creating a database using MySQL server
Table creation
Inserting data into the Table
Updating data in the DB
Formatting Output into a pandas DataFrame 
Using JOIN to combine the record of two or more Tables
Delete record from Table
Completely delete table using DROP
Creating Records from Lists exist data

You can get more details on this tutorial by reading checking on the two links below:
https://www.freecodecamp.org/news/connect-python-with-sql/
https://www.kaggle.com/code/ganu1899/sql-with-python

"""

'This is data analysis using Structured Query Language (SQL)\nSQL is a tool for every data scientist for getting access to data, \nand for advance data analysis.\nDesigning the data stracture properly is very important, \nit is adversable to devide data to many tables, and not stored all in one place\n\nUsing build SQL using BigQuery\nBigQuery is a Google Cloud product for storing and accessing very large databases\n\nThe notebook will cover:\nTable creation\ninserting / updating data in the DB\nfunctions - gets a value as an input, and returns manipulation of that value '

In [86]:
# import our bigquery
import numpy as np
import pandas as pd
import os
import sqlite3
import matplotlib.pyplot as plt
import mysql.connector
from mysql.connector import Error

In [87]:
#Create Connection for mysql


def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            auth_plugin='mysql_native_password',
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

#Run the function to Connect to the mysql server
connection = create_server_connection("localhost", YourUserName, YourPassword)

MySQL Database connection successful


In [88]:
#create a new database on our server, You don't need to run this all the time 
#if the database already exist

def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

#Next we define a query that use SQL to create the database and call the function:
create_database_query = "CREATE DATABASE IF NOT EXISTS school"
create_database(connection, create_database_query)

"""
You can delete the created daabase on your mysql server command prompt using the command below. 
"DROP DATABASE IF EXISTS database_name;"
But be warn that this is irreversible once deleted
"""

Database created successfully


'\nYou can delete the created daabase on your mysql server command prompt using the command below. \n"DROP DATABASE IF EXISTS database_name;"\nBut be warn that this is irreversible once deleted\n'

In [89]:
"""
Connecting to the Database
After we have created a database in MySQL Server, we can modify our create_server_connection 
function to connect directly to the  database we have created.

This is the code you need to run all the time you want to connect to already created database
"""
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection


In [91]:
"""
Creating a Query Execution Function
A  query execution function is an extremely vital one, it takes our SQL queries, stored in Python as 
strings, and pass them to the cursor.execute() method to execute them on the server.
"""
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")



In [92]:
"""
Creating Tables

Take note of the SQL commands, you can use either "CREATE TABLE" or "CREATE TABLE IF NOT EXISTS"
I love using "CREATE TABLE IF NOT EXISTS" instead

In creating Table, you can also use auto primary key "INT AUTO_INCREMENT PRIMARY KEY" instead of the key 
you have to manually supply by using "INT PRIMARY KEY" that is used below
"""

#Let's start with our Teacher table:
create_teacher_table = """
CREATE TABLE IF NOT EXISTS teacher (
  teacher_id INT PRIMARY KEY,
  first_name VARCHAR(40) NOT NULL,
  last_name VARCHAR(40) NOT NULL,
  language_1 VARCHAR(3) NOT NULL,
  language_2 VARCHAR(3),
  dob DATE,
  tax_id INT UNIQUE,
  phone_no VARCHAR(20)
  );
 """


execute_query(connection, create_teacher_table) # Execute our defined query



MySQL Database connection successful
Query successful


In [93]:
"""
You can verify if the table is created by runing this command on the MySQL server command line client
Open a terminal and type "mysql -u root -p databasename;", and after connection, type the following
command "SHOW TABLES;"

You can also delete the tableusing the command: SET FOREIGN_KEY_CHECKS=0; DROP TABLE teacher; SET FOREIGN_KEY_CHECKS=1;"
Take note that the command "SET FOREIGN_KEY_CHECKS=0" in the list of trhe commands above is to avoid errors
that can come up due to foreign key reference that link the tables together

#The alternative way, is to use the command below
Command to check the list of existing tables in the database

"""
mycursor = connection.cursor()
mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)

('teacher',)


In [94]:
"""
Now let's create the remaining tables.
"""

create_client_table = """
CREATE TABLE IF NOT EXISTS client (
  client_id INT PRIMARY KEY,
  client_name VARCHAR(40) NOT NULL,
  address VARCHAR(60) NOT NULL,
  industry VARCHAR(20)
);
 """

create_participant_table = """
CREATE TABLE IF NOT EXISTS participant (
  participant_id INT PRIMARY KEY,
  first_name VARCHAR(40) NOT NULL,
  last_name VARCHAR(40) NOT NULL,
  phone_no VARCHAR(20),
  client INT
);
"""

create_course_table = """
CREATE TABLE IF NOT EXISTS course (
  course_id INT PRIMARY KEY,
  course_name VARCHAR(40) NOT NULL,
  language VARCHAR(3) NOT NULL,
  level VARCHAR(2),
  course_length_weeks INT,
  start_date DATE,
  in_school BOOLEAN,
  teacher INT,
  client INT
);
"""

execute_query(connection, create_client_table)
execute_query(connection, create_participant_table)
execute_query(connection, create_course_table)


Query successful
Query successful
Query successful


In [95]:
"""
Creating the relationship between the Table is very important.
Now we want to define the relationships between them and create one
more table to handle the many-to-many relationship between the participant and course tables
"""
alter_participant = """
ALTER TABLE participant
ADD FOREIGN KEY(client)
REFERENCES client(client_id)
ON DELETE SET NULL;
"""

alter_course = """
ALTER TABLE course
ADD FOREIGN KEY(teacher)
REFERENCES teacher(teacher_id)
ON DELETE SET NULL;
"""

alter_course_again = """
ALTER TABLE course
ADD FOREIGN KEY(client)
REFERENCES client(client_id)
ON DELETE SET NULL;
"""

create_takescourse_table = """
CREATE TABLE IF NOT EXISTS takes_course (
  participant_id INT,
  course_id INT,
  PRIMARY KEY(participant_id, course_id),
  FOREIGN KEY(participant_id) REFERENCES participant(participant_id) ON DELETE CASCADE,
  FOREIGN KEY(course_id) REFERENCES course(course_id) ON DELETE CASCADE
);
"""


execute_query(connection, alter_participant)
execute_query(connection, alter_course)
execute_query(connection, alter_course_again)
execute_query(connection, create_takescourse_table)


Query successful
Query successful
Query successful
Query successful


In [96]:
"""
Populating the Tables
The next step is to add some records to the tables
"""
pop_teacher = """
INSERT INTO teacher VALUES 
(1,  'James', 'Smith', 'ENG', NULL, '1985-04-20', 12345, '+491774553676'),
(2, 'Stefanie',  'Martin',  'FRA', NULL,  '1970-02-17', 23456, '+491234567890'), 
(3, 'Steve', 'Wang',  'MAN', 'ENG', '1990-11-12', 34567, '+447840921333'),
(4, 'Friederike',  'Müller-Rossi', 'DEU', 'ITA', '1987-07-07',  45678, '+492345678901'),
(5, 'Isobel', 'Ivanova', 'RUS', 'ENG', '1963-05-30',  56789, '+491772635467'),
(6, 'Niamh', 'Murphy', 'ENG', 'IRI', '1995-09-08',  67890, '+491231231232');
"""

execute_query(connection, pop_teacher)


Query successful


In [97]:
"""Reading Data from the Table
AT this tage we make use of  cursor.fetchall() instead of cursor.commit()
for the function"""
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [98]:
"""
You can check the content of the table using the command "SELECT * FROM teacher;" on the mysql server

Alternatively, use the function to fetch every content of the table "teacher"

Here I will format the Output into a pandas DataFrame
"""
q1 = """
SELECT *
FROM teacher;
"""

results1 = read_query(connection, q1)

# Formatting Output into a pandas DataFrame
from_db_teacher = []

for result in results1:
  result = list(result)
  from_db_teacher.append(result)

columns = ["teacher_id", "firstname", "surname", "first-language", "second-language", 
           "Date-of-birth", "tax-id", "phone-number"]
df1 = pd.DataFrame(from_db_teacher, columns=columns)
df1.head(20)

,teacher_id,firstname,surname,first-language,second-language,Date-of-birth,tax-id,phone-number
0,1,James,Smith,ENG,None,1985-04-20,12345,+491774553676
1,2,Stefanie,Martin,FRA,None,1970-02-17,23456,+491234567890
2,3,Steve,Wang,MAN,ENG,1990-11-12,34567,+447840921333
3,4,Friederike,Müller-Rossi,DEU,ITA,1987-07-07,45678,+492345678901
4,5,Isobel,Ivanova,RUS,ENG,1963-05-30,56789,+491772635467
5,6,Niamh,Murphy,ENG,IRI,1995-09-08,67890,+491231231232


In [99]:
#Populate other tables

pop_client = """
INSERT INTO client VALUES
(101, 'Big Business Federation', '123 Falschungstraße, 10999 Berlin', 'NGO'),
(102, 'eCommerce GmbH', '27 Ersatz Allee, 10317 Berlin', 'Retail'),
(103, 'AutoMaker AG',  '20 Künstlichstraße, 10023 Berlin', 'Auto'),
(104, 'Banko Bank',  '12 Betrugstraße, 12345 Berlin', 'Banking'),
(105, 'WeMoveIt GmbH', '138 Arglistweg, 10065 Berlin', 'Logistics');
"""

pop_participant = """
INSERT INTO participant VALUES
(101, 'Marina', 'Berg','491635558182', 101),
(102, 'Andrea', 'Duerr', '49159555740', 101),
(103, 'Philipp', 'Probst',  '49155555692', 102),
(104, 'René',  'Brandt',  '4916355546',  102),
(105, 'Susanne', 'Shuster', '49155555779', 102),
(106, 'Christian', 'Schreiner', '49162555375', 101),
(107, 'Harry', 'Kim', '49177555633', 101),
(108, 'Jan', 'Nowak', '49151555824', 101),
(109, 'Pablo', 'Garcia',  '49162555176', 101),
(110, 'Melanie', 'Dreschler', '49151555527', 103),
(111, 'Dieter', 'Durr',  '49178555311', 103),
(112, 'Max', 'Mustermann', '49152555195', 104),
(113, 'Maxine', 'Mustermann', '49177555355', 104),
(114, 'Heiko', 'Fleischer', '49155555581', 105);
"""

pop_course = """
INSERT INTO course VALUES
(12, 'English for Logistics', 'ENG', 'A1', 10, '2020-02-01', TRUE,  1, 105),
(13, 'Beginner English', 'ENG', 'A2', 40, '2019-11-12',  FALSE, 6, 101),
(14, 'Intermediate English', 'ENG', 'B2', 40, '2019-11-12', FALSE, 6, 101),
(15, 'Advanced English', 'ENG', 'C1', 40, '2019-11-12', FALSE, 6, 101),
(16, 'Mandarin für Autoindustrie', 'MAN', 'B1', 15, '2020-01-15', TRUE, 3, 103),
(17, 'Français intermédiaire', 'FRA', 'B1',  18, '2020-04-03', FALSE, 2, 101),
(18, 'Deutsch für Anfänger', 'DEU', 'A2', 8, '2020-02-14', TRUE, 4, 102),
(19, 'Intermediate English', 'ENG', 'B2', 10, '2020-03-29', FALSE, 1, 104),
(20, 'Fortgeschrittenes Russisch', 'RUS', 'C1',  4, '2020-04-08',  FALSE, 5, 103);
"""

pop_takescourse = """
INSERT INTO takes_course VALUES
(101, 15),
(101, 17),
(102, 17),
(103, 18),
(104, 18),
(105, 18),
(106, 13),
(107, 13),
(108, 13),
(109, 14),
(109, 15),
(110, 16),
(110, 20),
(111, 16),
(114, 12),
(112, 19),
(113, 19);
"""

execute_query(connection, pop_client)
execute_query(connection, pop_participant)
execute_query(connection, pop_course)
execute_query(connection, pop_takescourse)

Query successful
Query successful
Query successful
Query successful


In [100]:
#Reading Data
#AT this tage we make use of  cursor.fetchall() instead of cursor.commit()
#for the function
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [101]:
#use the function to fetch every content of the table "teacher"
q1 = """
SELECT *
FROM teacher;
"""

results1 = read_query(connection, q1)

for result in results1:
  print(result)


(1, 'James', 'Smith', 'ENG', None, datetime.date(1985, 4, 20), 12345, '+491774553676')
(2, 'Stefanie', 'Martin', 'FRA', None, datetime.date(1970, 2, 17), 23456, '+491234567890')
(3, 'Steve', 'Wang', 'MAN', 'ENG', datetime.date(1990, 11, 12), 34567, '+447840921333')
(4, 'Friederike', 'Müller-Rossi', 'DEU', 'ITA', datetime.date(1987, 7, 7), 45678, '+492345678901')
(5, 'Isobel', 'Ivanova', 'RUS', 'ENG', datetime.date(1963, 5, 30), 56789, '+491772635467')
(6, 'Niamh', 'Murphy', 'ENG', 'IRI', datetime.date(1995, 9, 8), 67890, '+491231231232')


In [102]:
# Formatting Output into a pandas DataFrame
from_db_teacher = []

for result in results1:
  result = list(result)
  from_db_teacher.append(result)

columns = ["teacher_id", "firstname", "surname", "first-language", "second-language", 
           "Date-of-birth", "tax-id", "phone-number"]
df1 = pd.DataFrame(from_db_teacher, columns=columns)
df1.head()

,teacher_id,firstname,surname,first-language,second-language,Date-of-birth,tax-id,phone-number
0,1,James,Smith,ENG,None,1985-04-20,12345,+491774553676
1,2,Stefanie,Martin,FRA,None,1970-02-17,23456,+491234567890
2,3,Steve,Wang,MAN,ENG,1990-11-12,34567,+447840921333
3,4,Friederike,Müller-Rossi,DEU,ITA,1987-07-07,45678,+492345678901
4,5,Isobel,Ivanova,RUS,ENG,1963-05-30,56789,+491772635467


In [103]:
q2 = """
SELECT course.course_id, course.course_name, course.language, client.client_name, client.address
FROM course
JOIN client
ON course.client = client.client_id
WHERE course.in_school = FALSE;
"""

results = read_query(connection, q2)

for result in results:
  print(result)

(13, 'Beginner English', 'ENG', 'Big Business Federation', '123 Falschungstraße, 10999 Berlin')
(14, 'Intermediate English', 'ENG', 'Big Business Federation', '123 Falschungstraße, 10999 Berlin')
(15, 'Advanced English', 'ENG', 'Big Business Federation', '123 Falschungstraße, 10999 Berlin')
(17, 'Français intermédiaire', 'FRA', 'Big Business Federation', '123 Falschungstraße, 10999 Berlin')
(19, 'Intermediate English', 'ENG', 'Banko Bank', '12 Betrugstraße, 12345 Berlin')
(20, 'Fortgeschrittenes Russisch', 'RUS', 'AutoMaker AG', '20 Künstlichstraße, 10023 Berlin')


In [104]:
# Formatting Output into a pandas DataFrame
from_db_course = []

for result in results:
  result = list(result)
  from_db_course.append(result)


columns = ["course_id", "course_name", "language", "client_name", "address"]
df = pd.DataFrame(from_db_course, columns=columns)
df.head()

,course_id,course_name,language,client_name,address
0,13,Beginner English,ENG,Big Business Federation,"123 Falschungstraße, 10999 Berlin"
1,14,Intermediate English,ENG,Big Business Federation,"123 Falschungstraße, 10999 Berlin"
2,15,Advanced English,ENG,Big Business Federation,"123 Falschungstraße, 10999 Berlin"
3,17,Français intermédiaire,FRA,Big Business Federation,"123 Falschungstraße, 10999 Berlin"
4,19,Intermediate English,ENG,Banko Bank,"12 Betrugstraße, 12345 Berlin"


In [105]:
#use the function to fetch every content of the table "client"
q3 = """
SELECT *
FROM client;
"""

results3 = read_query(connection, q3)

for result in results3:
  print(result)



(101, 'Big Business Federation', '123 Falschungstraße, 10999 Berlin', 'NGO')
(102, 'eCommerce GmbH', '27 Ersatz Allee, 10317 Berlin', 'Retail')
(103, 'AutoMaker AG', '20 Künstlichstraße, 10023 Berlin', 'Auto')
(104, 'Banko Bank', '12 Betrugstraße, 12345 Berlin', 'Banking')
(105, 'WeMoveIt GmbH', '138 Arglistweg, 10065 Berlin', 'Logistics')


In [106]:
# Formatting Output into a pandas DataFrame
from_db_client = []

for result in results3:
  result = list(result)
  from_db_client.append(result)


columns = ["client_id", "client_name", "address", "industry"]
df3 = pd.DataFrame(from_db_client, columns=columns)
df3.head()

,client_id,client_name,address,industry
0,101,Big Business Federation,"123 Falschungstraße, 10999 Berlin",NGO
1,102,eCommerce GmbH,"27 Ersatz Allee, 10317 Berlin",Retail
2,103,AutoMaker AG,"20 Künstlichstraße, 10023 Berlin",Auto
3,104,Banko Bank,"12 Betrugstraße, 12345 Berlin",Banking
4,105,WeMoveIt GmbH,"138 Arglistweg, 10065 Berlin",Logistics


In [107]:
#Updating Records
update = """
UPDATE client 
SET address = '23 Fingiertweg, 14534 Berlin' 
WHERE client_id = 101;
"""

execute_query(connection, update)


Query successful


In [108]:
#Prevent SQL Injection
#It is considered a good practice to escape the values of any query, also in update statements.
#This is to prevent SQL injections, which is a common web hacking technique to destroy or misuse your database.
#THis can be done using placeholder %s 

#Updating Records
update = """
UPDATE client 
SET address = %s 
WHERE client_id = %s;
"""

val = ("23 Peacful, 14534 Berlin", "101")


#Let modify the previous execute_query2 to include the placeholder variable
def execute_query2(connection, query, variables):
    cursor = connection.cursor()
    try:
        cursor.execute(query, variables)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")
        
        
execute_query2(connection, update, val)



Query successful


In [109]:
#Creating Records from Lists of data already exist and add it to
#the Table
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [110]:
sql = '''
    INSERT INTO teacher (teacher_id, first_name, last_name, language_1, language_2, dob, tax_id, phone_no) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    '''
    
val = [
    (7, 'Hank', 'Dodson', 'ENG', None, '1991-12-23', 11111, '+491772345678'), 
    (8, 'Sue', 'Perkins', 'MAN', 'ENG', '1976-02-02', 22222, '+491443456432')
]

execute_list_query(connection, sql, val)

Query successful


In [111]:
#To turn existing table values into list for it to be added to existing table
#Let turn the whole table of teacher to list

q1 = """
SELECT *
FROM teacher;
"""

results1 = read_query(connection, q1)

# Formatting Output into a pandas DataFrame
from_db_teacher = []

for result in results1:
  result = list(result)
  from_db_teacher.append(result)

columns = ["teacher_id", "firstname", "surname", "first-language", "second-language", 
           "Date-of-birth", "tax-id", "phone-number"]
df1 = pd.DataFrame(from_db_teacher, columns=columns)


vals=df1.values
vals_list= vals.tolist()
vals_list

[[1,
  'James',
  'Smith',
  'ENG',
  None,
  datetime.date(1985, 4, 20),
  12345,
  '+491774553676'],
 [2,
  'Stefanie',
  'Martin',
  'FRA',
  None,
  datetime.date(1970, 2, 17),
  23456,
  '+491234567890'],
 [3,
  'Steve',
  'Wang',
  'MAN',
  'ENG',
  datetime.date(1990, 11, 12),
  34567,
  '+447840921333'],
 [4,
  'Friederike',
  'Müller-Rossi',
  'DEU',
  'ITA',
  datetime.date(1987, 7, 7),
  45678,
  '+492345678901'],
 [5,
  'Isobel',
  'Ivanova',
  'RUS',
  'ENG',
  datetime.date(1963, 5, 30),
  56789,
  '+491772635467'],
 [6,
  'Niamh',
  'Murphy',
  'ENG',
  'IRI',
  datetime.date(1995, 9, 8),
  67890,
  '+491231231232'],
 [7,
  'Hank',
  'Dodson',
  'ENG',
  None,
  datetime.date(1991, 12, 23),
  11111,
  '+491772345678'],
 [8,
  'Sue',
  'Perkins',
  'MAN',
  'ENG',
  datetime.date(1976, 2, 2),
  22222,
  '+491443456432']]

In [112]:
#To join two tables together, let extract information fromthe client
#and course table that we have created to form a new data table

#First use INNER join
q4 = """SELECT 
  client.client_id AS client_id,
  client.client_name AS user, 
  course.course_name AS favorite_subject 
  FROM client 
  INNER JOIN course ON client.client_id = course.client;"""


results4 = read_query(connection, q4)


from_db = []

for result in results4:
    result = list(result)
    from_db.append(result)


columns = ["client_id", "user", "favorite_subject"]
df4 = pd.DataFrame(from_db, columns=columns)
df4.head(10)


,client_id,user,favorite_subject
0,101,Big Business Federation,Beginner English
1,101,Big Business Federation,Intermediate English
2,101,Big Business Federation,Advanced English
3,101,Big Business Federation,Français intermédiaire
4,102,eCommerce GmbH,Deutsch für Anfänger
5,103,AutoMaker AG,Mandarin für Autoindustrie
6,103,AutoMaker AG,Fortgeschrittenes Russisch
7,104,Banko Bank,Intermediate English
8,105,WeMoveIt GmbH,English for Logistics


In [113]:
#First use LEFT join
q4 = """SELECT 
  client.client_id AS client_id,
  client.client_name AS user, 
  course.course_name AS favorite_subject 
  FROM client 
  LEFT JOIN course ON client.client_id = course.client;"""


results4 = read_query(connection, q4)


from_db = []

for result in results4:
    result = list(result)
    from_db.append(result)


columns = ["client_id", "user", "favorite_subject"]
df4 = pd.DataFrame(from_db, columns=columns)
df4.head(10)

,client_id,user,favorite_subject
0,101,Big Business Federation,Beginner English
1,101,Big Business Federation,Intermediate English
2,101,Big Business Federation,Advanced English
3,101,Big Business Federation,Français intermédiaire
4,102,eCommerce GmbH,Deutsch für Anfänger
5,103,AutoMaker AG,Mandarin für Autoindustrie
6,103,AutoMaker AG,Fortgeschrittenes Russisch
7,104,Banko Bank,Intermediate English
8,105,WeMoveIt GmbH,English for Logistics


In [114]:
#First use RIGHT join
q4 = """SELECT 
  client.client_id AS client_id,
  client.client_name AS user, 
  course.course_name AS favorite_subject 
  FROM client 
  RIGHT JOIN course ON client.client_id = course.client;"""


results4 = read_query(connection, q4)


from_db = []

for result in results4:
    result = list(result)
    from_db.append(result)


columns = ["client_id", "user", "favorite_subject"]
df4 = pd.DataFrame(from_db, columns=columns)
df4.head(10)

,client_id,user,favorite_subject
0,105,WeMoveIt GmbH,English for Logistics
1,101,Big Business Federation,Beginner English
2,101,Big Business Federation,Intermediate English
3,101,Big Business Federation,Advanced English
4,103,AutoMaker AG,Mandarin für Autoindustrie
5,101,Big Business Federation,Français intermédiaire
6,102,eCommerce GmbH,Deutsch für Anfänger
7,104,Banko Bank,Intermediate English
8,103,AutoMaker AG,Fortgeschrittenes Russisch


In [82]:

#Deleting record: Care must be taking because any record delete is irrecovable again
delete_table_record = """
DELETE FROM course 
WHERE course_id = 2034;
"""


execute_query(connection, delete_table_record)


Query successful


In [81]:
#If you are interested in deleting a whole Table, you can use drop table command
drop_table = """
DROP TABLE IF EXISTS  teacher
"""

execute_query(connection, drop_table)

"""
If the above command gives a foreign key constraint error, you can delete on mysql server 
First start the server with: mysql -u root -p yourdatabase
You can also delete the table using the command: 
SET FOREIGN_KEY_CHECKS=0; DROP TABLE teacher; SET FOREIGN_KEY_CHECKS=1;"
"""

Error: '3730 (HY000): Cannot drop table 'teacher' referenced by a foreign key constraint 'course_ibfk_1' on table 'course'.'


In [115]:
#use the function to fetch every content of the table "teacher"
q1 = """
SELECT *
FROM teacher;
"""

results1 = read_query(connection, q1)

# Formatting Output into a pandas DataFrame
from_db_teacher = []

for result in results1:
  result = list(result)
  from_db_teacher.append(result)

columns = ["teacher_id", "firstname", "surname", "first-language", "second-language", 
           "Date-of-birth", "tax-id", "phone-number"]
df1 = pd.DataFrame(from_db_teacher, columns=columns)
df1.head(20)

,teacher_id,firstname,surname,first-language,second-language,Date-of-birth,tax-id,phone-number
0,1,James,Smith,ENG,None,1985-04-20,12345,+491774553676
1,2,Stefanie,Martin,FRA,None,1970-02-17,23456,+491234567890
2,3,Steve,Wang,MAN,ENG,1990-11-12,34567,+447840921333
3,4,Friederike,Müller-Rossi,DEU,ITA,1987-07-07,45678,+492345678901
4,5,Isobel,Ivanova,RUS,ENG,1963-05-30,56789,+491772635467
5,6,Niamh,Murphy,ENG,IRI,1995-09-08,67890,+491231231232
6,7,Hank,Dodson,ENG,None,1991-12-23,11111,+491772345678
7,8,Sue,Perkins,MAN,ENG,1976-02-02,22222,+491443456432
